<a href="https://colab.research.google.com/github/numediart/ronde-nuit/blob/master/ronde_nuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ENG] Welcome to "La Ronde de Nuit" Colaboratory

This file was created to run python scripts that are outside of the demonstration. These scripts are developed for:
  * connecting to an IRC chat to collect a new database
  * convert a file from JSON (used for previous scripts) to CSV (for readability)
  * analyze a file with a sentiment analysis model
  * retrain a sentiment analysis model using new data

# [FR] Bienvenue sur le Colaboratory de "La Ronde de Nuit"

Ce fichier a été créé pour lancer les scripts python qui sont extérieur à la démonstration. Ces scripts sont utiles pour :
  * se connecter à un chat IRC pour en extraire une nouvelle base de données
  * convertir un fichier du format JSON (utilisé par les précédents scripts) vers le format CSV (plus lisible)
  * analyser un fichier avec un modèle de sentiment analysis (analyse de la polarité)
  * réentrainer un modèle de sentiment analysis avec de nouvelles données


## [ENG] 1. General setup

This is used to import the code and install usefull packages for the scripts.
Everything has to be run before the scripts and in order.

## [FR] 1. Mise en place générale

C'est utilisé pour import le code et installer les packages nécessaire pour utiliser les scripts.
Tout doit être lancé avant d'utiliser les scripts, et dans l'ordre.


In [ ]:
# Remove unnecessary folder 
!rm -rf sample_data/

# Download code
!git clone https://github.com/numediart/ronde-nuit.git

# Go to project folder
%cd ronde-nuit/

# Install required libraries
!pip install -r requirements.txt

Cloning into 'ronde-nuit'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 302 (delta 159), reused 234 (delta 92), pack-reused 0
Receiving objects: 100% (302/302), 6.37 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/ronde-nuit/ronde-nuit


## [ENG] 2. IRC chat connection

All you need to define is:
  * server, the ip address or DNS name to the server
  * port, the port to log on (usually 6667 is fine)
  * bot_name, the name the bot will have on the server (it is seen as someone logging to the server)
  * outputfile, the name of the file to save messages into. File must be a JSON
  * channel_i, the name of the ith channel to log into. Can be up to 6 but must be at least 1. 

## [FR] 2. Connexion à un chat IRC

Vous devez définir :
  * server, le nom ou adresse IP du serveur sur lequel se connecter
  * port, le port via lequel se connecter (généralement 6667 fonctionne)
  * bot_name, le nom du bot sur le serveur (qui est vu comme un client lambda)
  * outputfile, le nom du fichier dans lequel écrire les messages. Ce fichier doit être au format JSON.
  * channel_i, le nom du ième channel sur lequel se connecter. On peut en définir 6 au maximum et il en faut au moins 1.

In [ ]:
#@title IRC chat reading { run: "auto" }

server = 'irc.chaat.fr' #@param {type:"string"}
port = 6667  #@param {type:"number"}
bot_name = 'ronde' #@param {type:"string"}
outputfile = 'output.json' #@param {type:"string"}

#@markdown ---
#@markdown ### Channels

channel_0 = '#accueil' #@param {type:"string"}
channel_1 = '#maroc' #@param {type:"string"}
channel_2 = '#algerie' #@param {type:"string"}
channel_3 = '' #@param {type:"string"}
channel_4 = '' #@param {type:"string"}
channel_5 = '' #@param {type:"string"}

# Clean up channels
channels = [channel_0, channel_1, channel_2, 
            channel_3, channel_4, channel_5]
channels = [x for x in channels if not x == '']

In [ ]:
from src.connection import IrcBot

bot = IrcBot(server, channels, bot_name, port, outputfile)
bot.start()

Logged into #accueil@irc.chaat.fr:6667
Logged into #maroc@irc.chaat.fr:6667
Logged into #algerie@irc.chaat.fr:6667


KeyboardInterrupt: ignored

## [ENG] 3. File analysis

This will run a sentiment analysis model on a given file.
This requires:
  * srcfile, path to the file to analyze
  * model, the king of model to use (to be selected among a list of models)
  * threshold, a threshold to define the limit between POSITIVE/NEGATIVE and NEUTRAL. When confidence score of a sentiment is below the threshold, the label is set to NEUTRAL.

## [FR] 3. Analyse de fichier

Cette partie va lancer l'analyse d'un fichier donné avec un modèle de sentiment analysis.
Il faut fournir :
  * srcfile, le chemin vers le fichier à analyser
  * model, le type de modèle à utiliser (à sélectionner selon une liste de modèles prédéfinis)
  * threshold, un seuil pour définir la limite entre POSITIF/NEGATIF et NEUTRE. Quand le score de confiance du modèle est sous le seuil, le label devient NEUTRAL (neutre).


In [ ]:
#@title File analysis { run: "auto" }

versions = {
    'default': 1,
    'camembert': 0
}

srcfile = 'data/pre/chaat.json' #@param {type:"string"}
model = 'camembert' #@param ["default", "camembert"]
version = versions[model]
threshold = 0.66 #@param {type:"slider", min:0.0, max:1.0, step:0.01}

In [ ]:
from src.analysis import analyze_file

analyze_file(srcfile, version, threshold)

ValueError: ignored

## [ENG] 4. File conversion

This conversion is from JSON to CSV only.
You need to give **srcfile** path to the JSON file and **outputfile** path to the CSV file.

## [FR] 4. Conversion de fichier

Cette conversion se fait uniquement de JSON vers CSV.
Vous devez fournir le chemin vers le fichier JSON **srcfile** et le chemin vers le fichier CSV **outputfile**.

In [ ]:
#@title File conversion { run: "auto" }

versions = {
    'default': 1,
    'camembert': 0
}

srcfile = 'data/pre/chaat.json' #@param {type:"string"}
outputfile = 'data/pre/chaat.csv' #@param {type:"string"}

In [ ]:
from src.format import json2csv

json2csv(srcfile, outputfile)

## [ENG] 5. Dataset creation (for retraining)

This creates the dataset for retraining based on a CSV file of annotated sentences.

## [FR] 5. Création de la base de données (pour réentrainement)

Cette partie sert à créer une dataset pour du réentrainement basé sur un fichier CSV contenant des phrases annotées.

In [13]:
#@title Dataset creation { run: "auto" }

infolder = 'data/annotated' #@param {type:"string"}
outfolder = 'data/sorted' #@param {type:"string"}
test_ratio = 0.3 #@param {type:"slider", min:0.0, max:1.0, step:0.05}

In [14]:
from src.dataset import create_dataset

create_dataset(infolder, outfolder, test_ratio)

Positive ratio: 0.41784565916398714
Neutral ratio: 0.3463022508038585
Negative ratio: 0.21961414790996783


## [ENG] 6. Model retraining

## [FR] 6. Reentrainement du modèle

In [17]:
#@title Retraining folder { run: "auto" }

folder = 'data/sorted' #@param {type:"string"}

In [18]:
from src.retrain import retrain

retrain(folder)

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:114: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  FutureWarning,


TypeError: ignored